Analysis based on wards

In [1]:
from random import shuffle, randint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, LineString, mapping, shape
from descartes import PolygonPatch
import random
import fiona
import numpy as np
import csv
from fiona import collection

import geopandas as gpd
from geopandas.tools import sjoin # rtree index in-build, used with inner, intersection
import pandas as pd

from collections import defaultdict

In [9]:
def find_intersections(o):
    
    from collections import defaultdict

    paired_ind = [o.index_PP, o.index_WARD]

    d_over_ind = defaultdict(list)

    # creating a dictionary that has prescints as keys and associated small areas as values
    for i in range(len(paired_ind[0].values)):
        if not paired_ind[0].values[i]==paired_ind[1].values[i]: # it shows itself as intersection
            d_over_ind[paired_ind[0].values[i]].append(paired_ind[1].values[i])

    # get rid of the pol precincts with no small areas associated to them- not the most efficient way
    d_temp = {}
    for l in d_over_ind:
        if len(d_over_ind[l]):
            d_temp[l] = d_over_ind[l]

    return d_temp
      
def calculate_join_indices(g1_reind, g2_reind):
       
        # the indexing:
        out = sjoin(g1_reind, g2_reind, how ="inner", op = "intersects")
        
        out.drop('index_right', axis=1, inplace=True) # one index doubles, so we drop one
      
        # output retains only 1 area (left or right join), and gives no intersection area.
        # so we create an array with paired indices: police precincts with a ward
        # we use it in a loop in a function below
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    

In [145]:
def calculate_join(dict_over_ind, g1_reind, g2_reind):
        data_aggreg = []

        for index1, crim in g1_reind.iterrows():
            try:
                index1 = crim.index_PP
                
                wards_found = dict_over_ind[index1]
                for ward in range(len(wards_found)):
                    pom = g2_reind[g2_reind.index_WARD == wards_found[ward]]['geometry']        

                    area_int = pom.intersection(crim['geometry']).area.values[0]
                    area_crim = crim['geometry'].area

                    area_ward = pom.values[0].area

                    popu_ward = g2_reind[g2_reind.index_WARD == wards_found[ward]]['WARD_POP'].values[0]
                    murd_count = crim['murd_cnt']
                    pp_province = crim['province']
                    compnt_nm = crim['compnt_nm']
                    popu_frac = (area_int / area_ward) * popu_ward# fraction of the pop area contained inside the crim
                    
                    extra_info_col_names = ['MUNICNAME', 'PROVINCE', 'WARD_ID']
                        
                    extra_cols = g2_reind[g2_reind.index_WARD == wards_found[ward]][extra_info_col_names]#.filter(regex=("NAME"))

                    data_aggreg.append({'geometry': pom.intersection(crim['geometry']).values[0],\
                                        'index_PP': index1,'index_WARD': wards_found[ward] ,'area_pp': area_crim,\
                                        'area_ward': area_ward,'area_inter': area_int,\
                                        'popu_inter' : popu_frac, 'popu_ward': popu_ward,\
                                        'murd_cnt': murd_count,'province': pp_province,\
                                        'MUNICNAME': extra_cols.MUNICNAME.values[0],\
                                        'PROVINCE': extra_cols.PROVINCE.values[0],\
                                        'WARD_ID': extra_cols.WARD_ID.values[0],\
                                        'PP_name': compnt_nm})
            except:
                pass
            
        df_t = gpd.GeoDataFrame(data_aggreg,columns=['geometry', 'index_PP','index_WARD','area_pp',\
                                                     'area_ward','area_inter', 'popu_inter',\
                                                     'popu_ward', 'murd_cnt','province',\
                                                     'MUNICNAME','PROVINCE','WARD_ID', 'PP_name'])
        #df_t.to_file(out_name)
        return df_t

In [146]:
# this function adds the remaining columns, calculates fractions etc
def compute_final_col(df_temp):
    # add population data per PP, ratio, etc to the main table
    temp = df_temp.groupby(by=['index_PP'])['popu_inter'].sum().reset_index()

    data_with_population = pd.merge(df_temp, temp, on='index_PP', how='outer')\
            .rename(columns={'popu_inter_y':'popu_frac_per_pp', 'popu_inter_x':'popu_inter'})


    data_with_population['murd_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\
               * data_with_population['murd_cnt']
    data_mur_per_int = data_with_population.groupby(by=['index_WARD'])['murd_per_int'].sum().reset_index()

    data_mur_per_sal = data_mur_per_int.rename(columns={'murd_per_int':'murd_per_ward'})

    data_with_population['scal_fac_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\

    data_complete = pd.merge(data_with_population, data_mur_per_sal, on='index_WARD', how='outer')
           
    return data_complete

LOAD the data

In [13]:
ppSHP = 'shapefiles/updated/polPrec_murd2015_prov_aea.shp'
warSHP = '../maps/data/Wards2011_aea.shp'

geo_ward = gpd.GeoDataFrame.from_file(warSHP)
geo_pp = gpd.GeoDataFrame.from_file(ppSHP)

geo_pp_reind = geo_pp.reset_index().rename(columns={'index':'index_PP'})
geo_ward_reind = geo_ward.iloc[:,[2,3,7,8,9]].reset_index().rename(columns={'index':'index_WARD'})
    
dict_int = calculate_join_indices(geo_pp_reind,geo_ward_reind)

In [140]:
len(geo_ward)

4277

In [144]:
geo_pp_reind.head(n=2)

,index_PP,compnt_nm,create_dt,geometry,murd_cnt,province,version
0,0,botshabelo,20140207,POLYGON ((181098.5019976945 -3347212.875774949...,25,Free State,1.1.0
1,1,kubusiedrift,20140207,POLYGON ((278531.9598645054 -3691437.895521421...,0,0,1.1.0


In [147]:
from timeit import default_timer as timer

start = timer() 
df_int = calculate_join(dict_int ,geo_pp_reind, geo_ward_reind)
end = timer()
print("time: ", end - start)  

time:  219.41054996897583


In [148]:
# There are 101,546 intersections 
df_int_aea = compute_final_col(df_int) # add final calculations
df_int_aea.head(n=2)

,geometry,index_PP,index_WARD,area_pp,area_ward,area_inter,popu_inter,popu_ward,murd_cnt,province,MUNICNAME,PROVINCE,WARD_ID,PP_name,popu_frac_per_pp,murd_per_int,scal_fac_per_int,murd_per_ward
0,POLYGON ((174410.1256216596 -3347008.790421369...,0,1709,5.539637e+07,17799617.055663,2384089.253057,1127.912782,8421,25,Free State,Mangaung Metropolitan Municipality,Free State,49400028,botshabelo,38323.760000,0.735779,0.029431,5.588438
1,POLYGON ((174407.3500422863 -3347161.563272926...,26,1709,2.809204e+08,17799617.055663,15320373.488126,7248.069705,8421,28,Free State,Mangaung Metropolitan Municipality,Free State,49400028,boithuso,41997.394678,4.832346,0.172584,5.588438


In [157]:
df_int_aea['unit'] = 'ward'

In [158]:
df_int_aea.head()

,geometry,index_PP,index_WARD,area_pp,area_ward,area_inter,popu_inter,popu_ward,murd_cnt,province,MUNICNAME,PROVINCE,WARD_ID,PP_name,popu_frac_per_pp,murd_per_int,scal_fac_per_int,murd_per_ward,unit
0,POLYGON ((174410.1256216596 -3347008.790421369...,0,1709,5.539637e+07,17799617.055663,2384089.253057,1127.912782,8421,25,Free State,Mangaung Metropolitan Municipality,Free State,49400028,botshabelo,38323.760000,0.735779,0.029431,5.588438,ward
1,POLYGON ((174407.3500422863 -3347161.563272926...,26,1709,2.809204e+08,17799617.055663,15320373.488126,7248.069705,8421,28,Free State,Mangaung Metropolitan Municipality,Free State,49400028,boithuso,41997.394678,4.832346,0.172584,5.588438,ward
2,"POLYGON ((168466.601577126 -3343706.638016352,...",549,1709,9.435366e+08,17799617.055663,95154.314480,45.017512,8421,16,Free State,Mangaung Metropolitan Municipality,Free State,49400028,selosesha,35458.969946,0.020313,0.001270,5.588438,ward
3,POLYGON ((174502.6049684629 -3347639.816318621...,0,1710,5.539637e+07,3710465.432456,401395.129342,791.114386,7313,25,Free State,Mangaung Metropolitan Municipality,Free State,49400029,botshabelo,38323.760000,0.516073,0.020643,4.864267,ward
4,POLYGON ((172122.5841625608 -3347157.112365411...,26,1710,2.809204e+08,3710465.432456,3309070.303114,6521.885614,7313,28,Free State,Mangaung Metropolitan Municipality,Free State,49400029,boithuso,41997.394678,4.348193,0.155293,4.864267,ward


In [161]:
df_int_aea_waz = df_int_aea.iloc[:,[18,12,11,10,13,6,16,15]].sort('WARD_ID').reset_index().drop('index', axis=1)

In [136]:
df_int_aea_waz.to_csv('data/wards_pp_intersections.csv')

In [162]:
df_int_aea_waz.head()

,unit,WARD_ID,PROVINCE,MUNICNAME,PP_name,popu_inter,scal_fac_per_int,murd_per_int
0,ward,10101001,Western Cape,Matzikama Local Municipality,vredendal,1128.749984,0.124777,0.998215
1,ward,10101001,Western Cape,Matzikama Local Municipality,lutzville,2068.250016,0.635251,7.623011
2,ward,10101002,Western Cape,Matzikama Local Municipality,doring bay,2047.856952,0.622851,1.245702
3,ward,10101002,Western Cape,Matzikama Local Municipality,lutzville,476.251099,0.146278,1.755333
4,ward,10101002,Western Cape,Matzikama Local Municipality,vredendal,725.543920,0.080205,0.641638


Check whether the sums over provinces add up to the official/initial numbers:

In [96]:
df_int_aea.head()
data_prov = df_int_aea[['PROVINCE','province','murd_est_per_int']]
data_prov.groupby('province')['murd_est_per_int'].sum()

province
0                   0
Eastern Cape     3051
Free State        943
Gauteng          3671
Kwazulu/Natal    3759
Limpopo           777
Mpumalanga        831
North West        853
Northern Cape     411
Western Cape     3186
Name: murd_est_per_int, dtype: float64